In [ ]:

A. Données CSV depuis Inside Airbnb
Vérifier l'abonnement associé au CLI

%%bash

az account list --output table
Name                              CloudName    SubscriptionId                        TenantId                              State    IsDefault
--------------------------------  -----------  ------------------------------------  ------------------------------------  -------  -----------
Simplon - HDF - Data Engineer P1  AzureCloud   029b3537-0f24-400b-b624-6058a145efe1  a2e466aa-4f86-4545-b5b8-97da7c8febf3  Enabled  True
Simplon HDF Roubaix Dev IA P1     AzureCloud   111aaa69-41b9-4dfd-b6af-2ada039dd1ae  a2e466aa-4f86-4545-b5b8-97da7c8febf3  Enabled  False
Création du ressource group

%%bash

RG_NAME="RG_DBREAU"
LOCATION="FranceCentral"

az group create --name $RG_NAME --location $LOCATION
{
  "id": "/subscriptions/029b3537-0f24-400b-b624-6058a145efe1/resourceGroups/RG_DBREAU",
  "location": "francecentral",
  "managedBy": null,
  "name": "RG_DBREAU",
  "properties": {
    "provisioningState": "Succeeded"
  },
  "tags": null,
  "type": "Microsoft.Resources/resourceGroups"
}
Data Lake
Création du data lake storage gen 2

%%bash

RG_NAME="RG_DBREAU"

STORAGE_ACCOUNT_NAME="adlsbreau"
LOCATION="FranceCentral"

if $(az storage account check-name --name $STORAGE_ACCOUNT_NAME --query "nameAvailable" -o tsv); then
    az storage account create \
      --name $STORAGE_ACCOUNT_NAME \
      --resource-group $RG_NAME \
      --location $LOCATION \
      --sku Standard_LRS \
      --kind StorageV2 \
      --hns true   # 🚨 important pour créer data lake et pas un storage account
fi
Création du conteneur dans le data lake

%%bash

RG_NAME="RG_DBREAU"
STORAGE_ACCOUNT_NAME="adlsbreau"

CONTAINER_NAME="data"

if ! az storage container show --name $CONTAINER_NAME --account-name $STORAGE_ACCOUNT_NAME --query "name" -o tsv &>/dev/null; then
    # Créer un conteneur dans le Data Lake Storage Gen 2
    az storage container create \
      --name $CONTAINER_NAME \
      --account-name $STORAGE_ACCOUNT_NAME \
      --auth-mode login
fi
{
  "created": true
}
Key Vault
Création du Key Vault

%%bash

RG_NAME="RG_DBREAU"

KEY_VAULT_NAME="keyvault-dbreau"  
LOCATION="FranceCentral"

if ! az keyvault show --name $KEY_VAULT_NAME --query "name" -o tsv &>/dev/null; then
    az keyvault create \
        --name $KEY_VAULT_NAME \
        --resource-group $RG_NAME \
        --location $LOCATION \
        --sku standard \
        --enable-rbac-authorization false
fi
Vérifier les ressources

%%bash

az resource list --resource-group RG_DBREAU --output table
Name             ResourceGroup    Location       Type                               Status
---------------  ---------------  -------------  ---------------------------------  --------
adlsbreau        RG_DBREAU        francecentral  Microsoft.Storage/storageAccounts
keyvault-dbreau  RG_DBREAU        francecentral  Microsoft.KeyVault/vaults
Service principal secondaire
Créer le service principal lié au key vault et stocker les credentials

%%bash 

RG_NAME="RG_DBREAU"
KEY_VAULT_NAME="keyvault-dbreau"

SP_NAME="sp-keyvault-dbreau" 

# Récupére les informations du sp
SP_DETAILS=$(az ad sp create-for-rbac \
    --name "$SP_NAME" \
    --query "{appId:appId, password:password, tenant:tenant}" \
    -o json)

# Reparties les informations à stocker dans des variables
SP_SECONDARY_APP_ID=$(echo $SP_DETAILS | jq -r '.appId')
SP_SECONDARY_PASSWORD=$(echo $SP_DETAILS | jq -r '.password')
SP_SECONDARY_TENANT_ID=$(echo $SP_DETAILS | jq -r '.tenant')

# Reporte dans le fichier .env
{
  echo "SP_SECONDARY_APP_ID=$SP_SECONDARY_APP_ID"
  echo "SP_SECONDARY_PASSWORD=$SP_SECONDARY_PASSWORD"
  echo "SP_SECONDARY_TENANT_ID=$SP_SECONDARY_TENANT_ID"
  echo ""
} >> ../.env
WARNING: Found an existing application instance: (id) 229a6846-daa9-45e2-91b0-7e49ca23d033. We will patch it.
WARNING: The output includes credentials that you must protect. Be sure that you do not include these credentials in your code or check the credentials into your source control. For more information, see https://aka.ms/azadsp-cli
Définir la stratégie de sécurité du service principal sur get

%%bash

KEY_VAULT_NAME="keyvault-dbreau"
SP_SECONDARY_APP_ID=$(grep -oP '^SP_SECONDARY_APP_ID=\K.*' ../.env)

az keyvault set-policy \
    --name $KEY_VAULT_NAME \
    --spn $SP_SECONDARY_APP_ID \
    --secret-permissions get > $(if [[ "$OSTYPE" == "msys" || "$OSTYPE" == "cygwin" ]]; then echo "nul"; else echo "/dev/null"; fi) 2>&1 #output silencieux sur tous les os
Service principal principal
Création du service principal principal

%%bash

RG_NAME="RG_DBREAU"
PRINCIPAL_NAME="dl-sas-read-dbreau" 
ROLE_NAME="Storage Blob Data Contributor" # ou "Storage Blob Delegator"

# Créer le Service Principal et récupérer les informations
SP_DETAILS=$(az ad sp create-for-rbac --name "$PRINCIPAL_NAME" --query "{appId:appId, password:password, tenant:tenant}" -o json)

# Récupérer les informations du Service Principal
SP_PRINCIPAL_APP_ID=$(echo $SP_DETAILS | jq -r '.appId')
SP_PRINCIPAL_PASSWORD=$(echo $SP_DETAILS | jq -r '.password')
SP_PRINCIPAL_TENANT_ID=$(echo $SP_DETAILS | jq -r '.tenant')

# Stocker dans le .env
{
  echo "SP_PRINCIPAL_APP_ID=$SP_PRINCIPAL_APP_ID"
  echo "SP_PRINCIPAL_PASSWORD=$SP_PRINCIPAL_PASSWORD"
  echo "SP_PRINCIPAL_TENANT_ID=$SP_PRINCIPAL_TENANT_ID"
  echo ""
} >> ../.env
WARNING: Found an existing application instance: (id) f94b0d64-9554-498a-8f43-28d0439ce222. We will patch it.
WARNING: The output includes credentials that you must protect. Be sure that you do not include these credentials in your code or check the credentials into your source control. For more information, see https://aka.ms/azadsp-cli
%%bash

RG_NAME="RG_DBREAU"
STORAGE_ACCOUNT_NAME="adlsbreau"  
# PRINCIPAL_NAME="dl-sas-read-dbreau" 
ROLE_NAME="Storage Blob Data Contributor" # ou "Storage Blob Delegator"
# KEY_VAULT_NAME="keyvault-dbreau" 

SP_PRINCIPAL_APP_ID=$(grep -oP '^SP_PRINCIPAL_APP_ID=\K.*' ../.env)

SUBSCRIPTION_ID=$(az account show --query "id" -o tsv)

# Assigner le rôle
az role assignment create \
    --assignee "$SP_PRINCIPAL_APP_ID" \
    --role "$ROLE_NAME" \
    --scope "/subscriptions/$SUBSCRIPTION_ID/resourceGroups/$RG_NAME/providers/Microsoft.Storage/storageAccounts/$STORAGE_ACCOUNT_NAME" \
    > $(if [[ "$OSTYPE" == "msys" || "$OSTYPE" == "cygwin" ]]; then echo "nul"; else echo "/dev/null"; fi) 2>&1